In [4]:
%matplotlib inline

import librosa
import soundfile as sf
import numpy as np
from hmmlearn import hmm
import matplotlib.pyplot as plt  
import os

STRIDE_SIZE = 10.
FRAME_SIZE = 25.
N_MFCC = 16

WINDOW = 3000
WINDOW_STRIDE = 30

def find_peaks(X, Y, iterations=1):
    out = list(zip(X, Y))
    for i in range(iterations):
        temp_out = []
        for j in range(len(out)):
            if (j-1) >= 0 and (j+1) < len(out):
                if out[j-1][1] < out[j][1] and out[j][1] < out[j+1][1]:
                    temp_out.append(out[j])
        out = temp_out
    return out

def truncate_peaks(peaks, thresh = 20):
    curr = peaks[0]
    keep = []
    for p in peaks:
        if p[0] - curr[0] >= 10:
            keep.append(curr)
        curr = p
    return keep
    
def window_number_to_sec(win_num, win_len = WINDOW):
    stamp = win_num + (win_len // 2)
    return (FRAME_SIZE + STRIDE_SIZE * (stamp - 1)) / 1000.

def get_mfcc_features(song_data, sample_rate, stride_size = STRIDE_SIZE, frame_size = FRAME_SIZE):
    mfccs = librosa.feature.mfcc(song_data, sample_rate, 
                                 n_mfcc=N_MFCC,
                                 hop_length=int(STRIDE_SIZE / 1000. * sample_rate), 
                                 n_fft=int(FRAME_SIZE / 1000. * sample_rate))
    return mfccs

def load_song(song_file):
    data, samplerate = sf.read(song_file)
    data = data[:, 0]
    mfccs = get_mfcc_features(data, samplerate)
    mfccs = np.asarray(mfccs).T
    return mfccs

def continuous_prob_estimate(mfccs):
    low_score = 1e9
    low_score_index = -1

    X = []
    Y = []
    for i in range(0, np.shape(mfccs)[0], WINDOW_STRIDE):
        if np.shape(mfccs)[0] - WINDOW <= i:
            break
        model = hmm.GaussianHMM(n_components=1	)
        model = model.fit(mfccs[i:i+WINDOW//2], [WINDOW//2])
        curr_score = model.score(mfccs[i+WINDOW//2:i+WINDOW], [len(mfccs[i+WINDOW//2:i+WINDOW])])
        X.append(window_number_to_sec(i))
        Y.append(curr_score * -1)
        if curr_score < low_score:
            low_score = curr_score
            low_score_index = i
    return X, Y
            
def make_peak_plot(X, Y):
    peaks = find_peaks(X, Y, iterations=3)
    trunc_peaks = truncate_peaks(peaks)
    plt.plot(X, Y)
    # for p in peaks:
    #     plt.axvline(x=p[0], color="red")
    for p in trunc_peaks:
        plt.axvline(x=p[0], color="red")

    plt.show()

In [6]:
from pathlib import Path
home = str(Path.home())
SONG_DIR = home + "/Downloads/songdata/songdata/"

results = []

for filename in os.listdir(SONG_DIR):
    mfccs = load_song(SONG_DIR + filename)
    X, Y = continuous_prob_estimate(mfccs)
    peaks = find_peaks(X, Y, iterations=3)
    trunc_peaks = truncate_peaks(peaks)
    print(filename,"\n", trunc_peaks)
    results.append((filename, trunc_peaks))
    

Axwell_Ingrosso-On_My_Way_Dave_Winnel_Remix.wav 
 [(57.915, 119695.03009149962), (101.715, 129256.15394450353), (141.015, 102127.12157366502), (188.415, 114628.36761185803), (216.615, 99500.37207539416), (248.415, 134593.5658925903)]
Bastille - Good Grief (Don Diablo Remix).wav 
 [(23.415, 108973.7290647585), (52.215, 99294.37844496059), (85.515, 99225.94958886865), (134.415, 105730.61295289642), (169.515, 99488.21792066089), (201.315, 138645.9386885502)]
Ariana Grande - No Tears Left To Cry (Dark Intensity Remix).wav 
 [(47.115, 107975.81431946083), (63.015, 99506.86303140932), (88.815, 101664.04889807818), (126.015, 107791.69085281854), (140.415, 100715.90275816072), (165.915, 101083.33611056523), (227.715, 99028.4847516799), (257.415, 97801.38110244565)]
Twenty One Pilots _ Stressed Out (Dave Winnel Remix).wav 
 [(58.215, 107469.01762136673), (76.515, 97076.4497506525), (113.115, 98118.77336051739), (165.915, 105603.18660984888), (196.515, 99853.16474639456)]
Halsey - Bad At Love (G

East & Young _ Elevate (Original Mix).wav 
 [(28.515, 99424.48421945417), (61.815, 136949.57313634676), (78.915, 103133.40930006589), (107.415, 107404.80980255725), (139.815, 121483.27391816169), (157.215, 97543.81981421082), (187.815, 104930.09863021265), (235.515, 139759.73596124316), (267.915, 105533.09982348204)]
Martin Garrix & Third Party - Lions In The Wild (Extended Mix).wav 
 [(29.115, 114078.83216515438), (50.715, 92975.6339525265), (96.015, 115530.51250644936), (124.815, 116905.88403417225), (148.215, 100531.9782749076), (194.115, 114845.57117582668), (224.415, 128976.63484572455)]
TieÌsto & Oliver Heldens - The Right Song (feat. Natalie La Rose).wav 
 [(16.215, 131590.6946534548), (62.115, 101697.32517722386), (93.315, 122946.02245083133), (108.615, 102256.24429806063), (123.615, 102477.13330484396), (154.815, 99099.96055385507)]
Calvin Harris - This Is What You Came For (feat. Rihanna).wav 
 [(45.315, 108075.00736178996), (76.515, 103326.74326950096), (123.015, 115196.734

RuntimeError: Error opening '/home/somil/Downloads/songdata/songdata/.DS_Store': File contains data in an unknown format.